# Modeling and Evaluation

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from google.colab import drive
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error, accuracy_score, classification_report, roc_auc_score, roc_curve, confusion_matrix, roc_auc_score, auc
from imblearn.pipeline import Pipeline
import multiprocessing as mp
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense
from statsmodels.stats.proportion import proportion_confint
from keras.wrappers.scikit_learn import KerasClassifier

warnings.filterwarnings("ignore")

In [ ]:
drive.mount('/content/drive')

In [ ]:
cd 'drive/My Drive'

In [ ]:
X_train = np.load("data/avazu_ctr/train.npy")
y_train = np.load("data/avazu_ctr/train_labels.npy")
X_test = np.load("data/avazu_ctr/test.npy")
y_test = np.load("data/avazu_ctr/test_labels.npy")

In [ ]:
print(X_test.shape)

## 1. Logistic Regression

We first create base logistic regression model to compare with our tuned one. For all the algorithms which we implemented, we have performed hyperparameter tuning using Grid Search CV

In [ ]:
model = LogisticRegressionCV(cv=5)
model.fit(X_train, y_train)
yhat = model.predict(X_test)
probs = model.predict_proba(X_test)

We first create a baseline Logistic Regression model with cross validation over 5 folds.

In [ ]:
accuracy = accuracy_score(y_test, yhat)
print("Model accuracy: {}".format(accuracy))

In [ ]:
param_grid = {
    'C':np.logspace(0, 4, 10),
    'penalty': ['l1', 'l2']
}
model = LogisticRegression()
model, yhat = optimize_hyperparameters(X_train, X_test, y_train, model, 
                                 param_grid, n_jobs=-1, cv=5, scoring_fit='roc_auc')

probs = model.predict_proba(X_test)
print(model.best_score_)
print(model.best_params_)

In [ ]:
accuracy = accuracy_score(y_test, yhat)
print("Model accuracy: {}".format(accuracy))

Our model gave an accuracy of 81.39%. However, the accuracy is just not enough and we will not calculate the confidence interval for our accuracy predictions as well as for our predicted probabilites

In [ ]:
def prediction_interval(accuracy, z, num_samples):
  interval = z * np.sqrt((accuracy * (1 - accuracy)) / num_samples)
  lower = accuracy - interval
  upper = accuracy + interval
  return interval, lower, upper

def regression_interval(actual_probs, pred_probs, z):

  sum_errs = np.sum((actual_probs - pred_probs)**2)
  stdev = np.sqrt(1/(len(actual_probs)-2) * sum_errs)
  conf = z * stdev
  return conf

We have defined 2 functions above, 1 for calculating the prediction confidence interval for the accuracy score and another for calculating the interval if a probability estimate is given. For both the cases we have a 95% confidence interval being calculated with a Z score of 1.96 as a constant.


In [ ]:
interval, lower, upper = prediction_interval(accuracy, 1.96, X_test.shape[0])
print("interval: {:.4f}, lower: {:.4f}, upper: {:.4f}".format(interval, lower, upper))

Hence our prediction has a confidence of 

In [ ]:
conf = regression_interval(probs[:,1], yhat, 1.96)
print("confidence: {:.4f}".format(conf))

In [ ]:
print(classification_report(y_test, yhat))

In [ ]:
conf_mat = confusion_matrix(y_test, yhat)
print(conf_mat)

In [ ]:
logit_roc_auc = roc_auc_score(y_test, yhat)
logit_roc_auc

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, probs[:,1])

In [ ]:
def plot_auc_roc(fpr, tpr):
  fig = px.area(
      x=fpr, y=tpr,
      title=f'ROC Curve (AUC={auc(fpr, tpr):.4f})',
      labels=dict(x='False Positive Rate', y='True Positive Rate'),
      width=700, height=500
  )
  fig.add_shape(
      type='line', line=dict(dash='dash'),
      x0=0, x1=1, y0=0, y1=1
  )

  fig.update_yaxes(scaleanchor="x", scaleratio=1)
  fig.update_xaxes(constrain='domain')
  fig.show()

In [ ]:
plot_auc_roc(fpr, tpr)

## 2. XGBoost Classifier

The choice of XGBoost was not random rather based on the following evidence. 
XGboost and GBM follows the principle of gradient boosting. They just differ internally as a model. Specifically, XGboost uses a more regularized model formalization to control over-fitting, which gives it better performance.
We decided to use XGBoost so that we can curb over-fitting inherently and also exploit for faster convergence by using all the resources available to us

In [ ]:
#Baseline model
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Model accuracy: {}".format(accuracy))

In [ ]:
def optimize_hyperparameters(X_train_data, X_test_data, y_train_data, 
                       model, param_grid, n_jobs, cv=10, scoring_fit='neg_mean_squared_error'):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=n_jobs, 
        scoring=scoring_fit,
        verbose=2
    )
    
    model = gs.fit(X_train_data, y_train_data)
    preds = model.predict(X_test_data)
    
    return model, preds

In [ ]:
param_grid = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(40,60,100),
    'reg_alpha':[1e-5, 1e-2, 0.1, 1],
    'gamma':[i/10.0 for i in range(0,5)]
}
model = XGBClassifier(max_depth=9, n_estimators=40, reg_alpha=0.1, gamma=0.3)
model, preds = optimize_hyperparameters(X_train, X_test, y_train, model, 
                                 param_grid, n_jobs=-1, cv=5, scoring_fit='roc_auc')

probs = model.predict_proba(X_test)
print(model.best_score_)
print(model.best_params_)

In [ ]:
model = XGBClassifier(max_depth=9, n_estimators=40, reg_alpha=0.1, gamma=0.3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
probs = model.predict_proba(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Model accuracy: {}".format(accuracy))

In [ ]:
interval, lower, upper = prediction_interval(accuracy, 1.96, X_test.shape[0])
print("interval: {:.4f}, lower: {:.4f}, upper: {:.4f}".format(interval, lower, upper))

In [ ]:
conf = regression_interval(probs[:,1], yhat, 1.96)
print("confidence: {:.4f}".format(conf))

In [ ]:
print(classification_report(y_test, yhat))

In [ ]:
conf_mat = confusion_matrix(y_test, yhat)
print(conf_mat)

In [ ]:
xgb_roc_auc = roc_auc_score(y_test, yhat)
xgb_roc_auc

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, probs[:,1])

In [ ]:
plot_auc_roc(fpr, tpr)

### 3. Multi-Layer Perceptron


In [ ]:
model = Sequential()
model.add(Dense(10, input_dim=15, activation='relu'))
model.add(Dense(7, input_dim=10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
optimizer = SGD(learning_rate=0.15, momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy','AUC'])
history = model.fit(X_train, y_train, epochs=50, batch_size=32)

In [ ]:
# make probability predictions with the model
probs = model.predict(X_test)

# make class predictions with the model
preds = model.predict_classes(X_test)

In [ ]:
accuracy = accuracy_score(y_test, preds)
print("Model accuracy: {}".format(accuracy))

In [ ]:
model = Sequential()
model.add(Dense(10, input_dim=15, activation='tanh'))
model.add(Dense(7, input_dim=10, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
optimizer = SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy','AUC'])
history = model.fit(X_train, y_train, epochs=100, batch_size=64)

In [ ]:
# make probability predictions with the model
probs = model.predict(X_test)

# make class predictions with the model
preds = model.predict_classes(X_test)

In [ ]:
accuracy = accuracy_score(y_test, preds)
print("Model accuracy: {}".format(accuracy))

In [ ]:
interval, lower, upper = prediction_interval(accuracy, 1.96, X_test.shape[0])
print("interval: {:.4f}, lower: {:.4f}, upper: {:.4f}".format(interval, lower, upper))

In [ ]:
print(classification_report(y_test, preds))

In [ ]:
conf_mat = confusion_matrix(y_test, preds)
print(conf_mat)

In [ ]:
mlp_roc_auc = roc_auc_score(y_test, preds)
mlp_roc_auc

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, probs)

In [ ]:
plot_auc_roc(fpr, tpr)

|Model|Baseline|Tuned|95% CI|lower|upper|AUC| 
|-----|--------|--------|------|-----|-----|---|
|Logistic|0.8140|0.8102|0.0054|0.8047|0.8156|0.6384|
|XGBoost|0.8042|0.7962|0.0056|0.7906|0.8017|0.7113|
|MLP|0.6994|0.7758|0.0058|0.7700|0.7816|0.6956|